# Analyse Correlations in the Dataset
## **A Notebook for Finding Correlations Across Demographic Categories and Wage**

In [1]:
import pandas as pd

In [2]:
us_personal_a = pd.read_csv('us_personal_sample/psam_pusa_sample.csv')
us_personal_b = pd.read_csv('us_personal_sample/psam_pusb_sample.csv')
us_personal_c = pd.read_csv('us_personal_sample/psam_pusc_sample.csv')
us_personal_d = pd.read_csv('us_personal_sample/psam_pusd_sample.csv')

In [3]:
total = pd.concat([us_personal_a, us_personal_b, us_personal_c, us_personal_b])
total.drop(columns='Unnamed: 0', inplace=True)

In [4]:
total

,RT,SERIALNO,DIVISION,SPORDER,PUMA,REGION,ST,ADJINC,PWGTP,AGEP,...,PWGTP71,PWGTP72,PWGTP73,PWGTP74,PWGTP75,PWGTP76,PWGTP77,PWGTP78,PWGTP79,PWGTP80
0,P,2014000763571,5,1,3107,3,12,1070673,11,53,...,15,10,10,22,12,21,3,3,13,11
1,P,2018HU1163498,5,1,6300,3,12,1013097,13,54,...,4,23,28,4,13,14,5,12,3,24
2,P,2016000964524,9,2,5904,4,6,1054346,27,54,...,10,50,26,27,9,30,7,8,45,29
3,P,2017000168429,1,1,104,1,9,1035838,13,68,...,25,5,13,15,26,22,16,3,25,21
4,P,2015000747570,5,1,6002,3,13,1061242,19,46,...,14,15,27,23,7,18,34,4,20,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34169,P,2016000772726,4,2,602,2,20,1054346,20,57,...,19,18,19,6,20,5,20,19,6,5
34170,P,2018HU0197938,5,1,1300,3,24,1013097,6,56,...,10,2,2,5,5,5,10,5,1,6
34171,P,2014001002072,1,1,2800,1,25,1070673,24,30,...,7,24,40,5,23,8,25,36,25,6
34172,P,2016000103446,4,1,1000,2,27,1054346,44,59,...,51,37,82,41,14,40,74,85,40,42
